In [167]:
import xarray as xr
import reverse_geocoder as rg
from tqdm import tqdm
import pandas as pd
import itertools
import numpy as np
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
import pickle as pkl

In [147]:
country_size={"DE":357600,"AT":83879,"SE":450295,"FR":543941,"IT":301958,"PL":312679,"ES":498485,"NL":41543,"BE":30528}

In [6]:
data = xr.open_dataset(filename_or_obj=r"C:\Daten\Foschung\RiskAware\Data\Weather\rainfall\gpcp_v01r03_daily_d20220101_c20220406.area-subset.72.-12.36.26.nc", engine='netcdf4')

In [42]:
# I need the countries that each of the longitude and latidude points belong to
# I will use the reverse_geocoder package for this

latitudes = data.latitude.values
longitudes = data.longitude.values
longitudes = [i if i<180 else i-360 for i in longitudes]

country={}
product=itertools.product(latitudes,longitudes)
for i,(lat,lon) in tqdm(enumerate(product),total=len(latitudes)*len(longitudes)):
    coordinates = (lat,lon)
    results = rg.search(coordinates)

    country[coordinates]=results[0]['cc']

100%|██████████| 1443/1443 [2:06:43<00:00,  5.27s/it]     


In [64]:
#get all entries of country where the value == "DE"
de = {k:v for k,v in country.items() if v=="DE"}

In [168]:
geocoding=pkl.load(open(r"country_dict.pkl","rb"))

In [169]:
country_precip={}
for i in range(0,31):
    if i<9:
        day = "0"+str(i+1)
    else:
        day = str(i+1)
    data= xr.open_dataset(filename_or_obj=r"C:\Daten\Foschung\RiskAware\Data\Weather\rainfall\gpcp_v01r03_daily_d202201"+day+"_c20220406.area-subset.72.-12.36.26.nc", engine='netcdf4')
    precip=pd.DataFrame(np.array(data.precip).reshape(37,39),index=latitudes,columns=longitudes)
    precip[precip<0]=0
    for country in country_size.keys():
        country_keys = [k for k,v in geocoding.items() if v==country]
        for key in country_keys:
            if country not in country_precip.keys():
                country_precip[country] = precip.loc[key]*71500*111000*(country_size[country]/(len(country_keys)*71.5*111))
            else:
                country_precip[country] += precip.loc[key]*71500*111000*(country_size[country]/(len(country_keys)*71.5*111))

In [170]:
country_precip

{'DE': np.float32(26808884000000.0),
 'AT': np.float32(4464687700000.0),
 'SE': np.float32(26253552000000.0),
 'FR': np.float32(29337798000000.0),
 'IT': np.float32(13553819000000.0),
 'PL': np.float32(22366025000000.0),
 'ES': np.float32(16993399000000.0),
 'NL': np.float32(2932931300000.0),
 'BE': np.float32(1971470000000.0)}

In [63]:
# save the country dictionary
# as a pickle
import pickle
with open('country_dict.pkl', 'wb') as f:
    pickle.dump(country, f)